In [ ]:
# import pandas as pd
# import requests

# # GitHub GraphQL API endpoint
# allUsers = pd.read_csv("allUsernames.csv")['userList']
# GITHUB_GRAPHQL_API = 'https://api.github.com/graphql'
# # Function to execute a GraphQL query
# def run_github_query(query, headers):
#     response = requests.post(GITHUB_GRAPHQL_API, json={'query': query}, headers=headers)
#     if response.status_code == 200:
#         return response.json()
#     else:
#         raise Exception("Query failed to run by returning code of {}. {}".format(response.status_code, query))

# # Your GitHub API token
# headers = {
#     "Authorization": "bearer " + os.environ['GITHUB_TOKEN'],
#     "Content-Type": "application/json"
# }


# def generate_query(username):
#     return {
#         "query": f"""
#             {{
#                 search(query: "is:pr is:public archived:false author:{username} -user:{username}", type: ISSUE, first: 100) {{
#                     issueCount
#                 }}
#             }}
#         """
#     }


# # List of usernames
# # get usernames from allUsernames.csv

# # List to store the results
# results = []
# # Loop through the usernames
# for username in allUsers:
#     # Generate the query
#     query = generate_query(username)
#     print('query:', query['query'])
#     response = run_github_query(query['query'], headers)
#     userPRs = response['data']['search']['issueCount']
#     # add to results
#     results.append({'username': username, 'userPRs': userPRs})

# # Create a DataFrame from the results
# pd.DataFrame(results).to_csv("userPRData.csv", index=False)


In [11]:
import pandas as pd
import requests
import os

# GitHub GraphQL API endpoint
allUsers = pd.read_csv("../../GHData_users_meta/usernameList.txt", header=None)[0].tolist()
GITHUB_GRAPHQL_API = 'https://api.github.com/graphql'

# Function to execute a GraphQL query
def run_github_query(query, headers):
    response = requests.post(GITHUB_GRAPHQL_API, json={'query': query}, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("Query failed to run by returning code of {}. {}".format(response.status_code, query))

# Your GitHub API token
headers = {
    "Authorization": "bearer " + os.environ['GITHUB_TOKEN'],
    "Content-Type": "application/json"
}

def generate_starred_repos_query(username):
    return {
        "query": f"""
            {{
                user(login: "{username}") {{
                    starredRepositories {{
                        totalCount
                    }}
                }}
            }}
        """
    }

# List to store the results
starred_repos_results = []
# Loop through the usernames
for username in allUsers:
    # Generate the query for forked repositories
    query = generate_starred_repos_query(username)
    response = run_github_query(query['query'], headers)
    if response['data']['user'] is not None:
        starred_repos_count = response['data']['user']['starredRepositories']['totalCount']
        starred_repos_results.append({'username': username, 'starredRepositories': starred_repos_count})
    else:
        starred_repos_results.append({'username': username, 'starredRepositories': 0})

# Create a DataFrame from the results
pd.DataFrame(starred_repos_results).to_csv("../../GHData_users_meta/userStarredReposData.csv", index=False)


In [12]:
def generate_forked_repos_query(username):
    return {
        "query": f"""
            {{
                user(login: "{username}") {{
                    repositories(isFork: true, first: 0) {{
                        totalCount
                    }}
                }}
            }}
        """
    }   

# List to store the results
forked_repos_results = []
# Loop through the usernames
for username in allUsers:
    # Generate the query for forked repositories
    query = generate_forked_repos_query(username)
    response = run_github_query(query['query'], headers)
    if response['data']['user']:
        forked_repos_count = response['data']['user']['repositories']['totalCount']
    else:
        # Handle cases where the user data might not be available
        forked_repos_count = 0  # You can choose to set this as 0 or handle it differently
        print(f"Data for user '{username}' not available or user does not exist.")
    # Add to results
    forked_repos_results.append({'username': username, 'forkedRepos': forked_repos_count})

# Create a DataFrame from the results
pd.DataFrame(forked_repos_results).to_csv("../../GHData_users_meta/userForkedReposData.csv", index=False)


Data for user 'machine-perception-robotics-group' not available or user does not exist.
Data for user 'MIV-Group' not available or user does not exist.
Data for user 'BlackyYen' not available or user does not exist.
Data for user 'Machine-Learning-And-AI-Projects' not available or user does not exist.
Data for user 'bigdatasciencegroup' not available or user does not exist.
Data for user 'delira-dev' not available or user does not exist.
Data for user 'MIVRC' not available or user does not exist.
Data for user 'clovaai' not available or user does not exist.
Data for user 'TodoListIOS' not available or user does not exist.
Data for user 'mmk-ai' not available or user does not exist.
Data for user 'CS4445-JasonDykstra' not available or user does not exist.
Data for user 'Jen-Jon' not available or user does not exist.
Data for user 'InformationRetrieval-Hongik' not available or user does not exist.
Data for user 'SaliencyDetection' not available or user does not exist.
Data for user '1zuu

In [ ]:

total_stars = sum(repo['stargazerCount'] for repo in response['data']['user']['repositories']['nodes'])
print("Total stars received:", total_stars)

In [13]:
def generate_stars_received_query(username, after_cursor=None):
    after_clause = f', after: "{after_cursor}"' if after_cursor else ''
    return {
        "query": f"""
            {{
                user(login: "{username}") {{
                    repositories(first: 100{after_clause}) {{
                        nodes {{
                            stargazerCount
                        }}
                        pageInfo {{
                            endCursor
                            hasNextPage
                        }}
                    }}
                }}
            }}
        """
    }

# List to store the results
stars_received_results = []
# Loop through the usernames
for username in allUsers:
    total_stars = 0
    has_next_page = True
    end_cursor = None

    while has_next_page:
        query = generate_stars_received_query(username, end_cursor)
        response = run_github_query(query['query'], headers)
        if response['data']['user'] and response['data']['user']['repositories']['nodes']:
            repos_info = response['data']['user']['repositories']
            total_stars += sum(repo['stargazerCount'] for repo in repos_info['nodes'])
            has_next_page = repos_info['pageInfo']['hasNextPage']
            end_cursor = repos_info['pageInfo']['endCursor'] if has_next_page else None
        else:
            # Handle cases where the user data might not be available or there are no repositories
            has_next_page = False
            print(f"Data for user '{username}' not available or user does not exist.")
    stars_received_results.append({'username': username, 'totalStarsReceived': total_stars})

pd.DataFrame(stars_received_results).to_csv("../../GHData_users_meta/userTotalStarsReceived.csv", index=False)

Data for user 'machine-perception-robotics-group' not available or user does not exist.
Data for user 'MIV-Group' not available or user does not exist.
Data for user 'BlackyYen' not available or user does not exist.
Data for user 'Machine-Learning-And-AI-Projects' not available or user does not exist.
Data for user 'bigdatasciencegroup' not available or user does not exist.
Data for user 'delira-dev' not available or user does not exist.
Data for user 'MIVRC' not available or user does not exist.
Data for user 'clovaai' not available or user does not exist.
Data for user 'TodoListIOS' not available or user does not exist.
Data for user 'mmk-ai' not available or user does not exist.
Data for user 'CS4445-JasonDykstra' not available or user does not exist.
Data for user 'Jen-Jon' not available or user does not exist.
Data for user 'InformationRetrieval-Hongik' not available or user does not exist.
Data for user 'SaliencyDetection' not available or user does not exist.
Data for user '1zuu